# 📍 Biomechanical Analysis - Joint Angles & Load Estimation

This notebook calculates joint angles (elbow, knee, hip) and estimates the biomechanical load on key body parts (arms, legs, hips, chest) using 2D pose keypoints extracted via YOLOv8.

**Assumptions**:
- Body weight: 60 kg
- Dumbbell weight: 5.5 kg in each hand
- Static poses only


# Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import os

# 1. Joint angle calculations (elbow, knee, hip)

Mount Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Define Google Drive save path
drive_folder = "/content/drive/MyDrive/SEE_Assessment"
os.makedirs(drive_folder, exist_ok=True)

 Load the 2D Keypoints CSV

In [4]:
# Load keypoints downloaded and saved from the pose estimation notebook
df_keypoints = pd.read_csv('/content/drive/MyDrive/SEE_Assessment/keypoints.csv')

# Preview keypoints
print("Loaded Keypoints DataFrame:")
df_keypoints.head()

Loaded Keypoints DataFrame:


,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y
0,jeffrey-f-lin-QXzHGRXyYHE-unsplash.jpg,3065.725098,1558.334961,3090.520508,1517.928101,3021.086426,1527.423340,0.000000,0.000000,2939.552490,...,2941.851562,2427.377930,3191.780029,2906.098633,2952.642578,2926.668701,3194.012695,3301.167725,2970.237793,3378.594482
1,lawrence-crayton-IsSxH3_6WlE-unsplash.jpg,2161.735352,1947.670654,2319.386719,1839.853394,2036.743774,1803.511963,2495.243896,2008.917969,1830.809082,...,1425.745239,4818.091797,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,anastase-maragos-iUzgePOoGko-unsplash.jpg,1859.201904,1812.019775,1921.308350,1760.410156,1823.065430,1765.839111,2068.910645,1827.672974,0.000000,...,1785.220215,3034.496094,2194.188965,3755.902100,1790.057617,3750.529541,2263.456299,4505.608398,1792.108276,4490.726562
3,alonso-reyes-0HlI76m4jxU-unsplash.jpg,3536.448242,1786.597412,3615.124023,1814.739624,3529.202393,1704.428955,3594.565430,1999.838745,3403.657471,...,2186.354492,2218.531494,2471.978516,3312.746582,1446.751099,2756.197510,1936.270752,3302.712646,920.611511,3400.001953
4,rezli-QDSAgKr1cvo-unsplash.jpg,3587.418457,1332.568359,3589.087646,1275.971436,3518.568848,1278.863525,0.000000,0.000000,3348.544678,...,3257.466797,2545.974609,3463.394531,3143.130859,3210.892578,3166.665771,3356.718506,3762.965088,3067.129150,3752.788818


Define Utility Function to Calculate Joint Angles

In [5]:
def calculate_angle(a, b, c):
    """
    Calculates the angle at point b (in degrees), given 3 points: a, b, c
    a, b, c: (x, y) coordinate tuples
    """
    a, b, c = np.array(a), np.array(b), np.array(c)

    # Vectors: BA and BC
    ba = a - b
    bc = c - b

    # Calculate cosine of angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))

    # Prevent numerical errors/ Handle possible float precision issues
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)

    # Compute angle in degrees
    angle = np.degrees(np.arccos(cosine_angle))
    return round(angle, 2)


Loop Over the Keypoints DataFrame and Calculate Angles

*  Left & Right Elbow Angle

*   Left & Right Knee Angle

*   Left & Right Hip Angle







In [6]:
# List to store angle results
angle_records = []

for _, row in df_keypoints.iterrows():
    img_name = row['image']

    # Helper function to get a joint's (x, y) coordinates
    def pt(joint): return (row[f"{joint}_x"], row[f"{joint}_y"])

    try:
        # Elbow: shoulder–elbow–wrist
        left_elbow_angle = calculate_angle(pt('left_shoulder'), pt('left_elbow'), pt('left_wrist'))
        right_elbow_angle = calculate_angle(pt('right_shoulder'), pt('right_elbow'), pt('right_wrist'))

        # Knee: hip–knee–ankle
        left_knee_angle = calculate_angle(pt('left_hip'), pt('left_knee'), pt('left_ankle'))
        right_knee_angle = calculate_angle(pt('right_hip'), pt('right_knee'), pt('right_ankle'))

        # Hip: shoulder–hip–knee
        left_hip_angle = calculate_angle(pt('left_shoulder'), pt('left_hip'), pt('left_knee'))
        right_hip_angle = calculate_angle(pt('right_shoulder'), pt('right_hip'), pt('right_knee'))

        angle_records.append({
            'image': img_name,
            'left_elbow_angle': left_elbow_angle,
            'right_elbow_angle': right_elbow_angle,
            'left_knee_angle': left_knee_angle,
            'right_knee_angle': right_knee_angle,
            'left_hip_angle': left_hip_angle,
            'right_hip_angle': right_hip_angle
        })

    except Exception as e:
        print(f"Error processing {img_name}: {e}")


<ipython-input-5-63f781ed648a>:13: RuntimeWarning: invalid value encountered in scalar divide
  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))


***Store*** Angles to DataFrame & ***Save*** Joint Angles

In [7]:
# -------------------------------
# ✅ Save Joint Angles
# -------------------------------
df_angles = pd.DataFrame(angle_records)
print("📐 Joint Angle Calculations (Degrees):")
display(df_angles.head())

# Save to /content/
df_angles.to_csv("/content/joint_angles.csv", index=False)
print("✅ Joint angles saved to /content/joint_angles.csv")

# Save to Google Drive
drive_angles_path = os.path.join(drive_folder, "joint_angles.csv")
df_angles.to_csv(drive_angles_path, index=False)
print(f"✅ Joint angles also saved to {drive_angles_path}")

📐 Joint Angle Calculations (Degrees):


,image,left_elbow_angle,right_elbow_angle,left_knee_angle,right_knee_angle,left_hip_angle,right_hip_angle
0,jeffrey-f-lin-QXzHGRXyYHE-unsplash.jpg,161.42,170.29,178.66,179.01,177.26,173.57
1,lawrence-crayton-IsSxH3_6WlE-unsplash.jpg,133.49,64.37,NaN,NaN,34.89,14.39
2,anastase-maragos-iUzgePOoGko-unsplash.jpg,164.96,177.25,179.94,179.77,166.39,176.94
3,alonso-reyes-0HlI76m4jxU-unsplash.jpg,163.17,162.08,87.95,165.27,100.92,176.31
4,rezli-QDSAgKr1cvo-unsplash.jpg,130.57,144.55,174.35,170.51,177.99,171.04


✅ Joint angles saved to /content/joint_angles.csv
✅ Joint angles also saved to /content/drive/MyDrive/SEE_Assessment/joint_angles.csv


# 2. Biomechanical load estimation (arms, legs, hips, chest)

Define Load Estimation Function

In [8]:
def estimate_load_distribution(row, body_weight=60, dumbbell_weight=5.5):
    """
    Estimate biomechanical load on body parts based on joint angles.
    Output loads are in kg.
    """
    # Extract angles
    le, re = row['left_elbow_angle'], row['right_elbow_angle']
    lk, rk = row['left_knee_angle'], row['right_knee_angle']
    lh, rh = row['left_hip_angle'], row['right_hip_angle']

    # ------------------ Arm Load ------------------
    def arm_load(angle):
        return dumbbell_weight * (1 + (180 - angle) / 90)  # more extension = more stress

    left_arm = arm_load(le)
    right_arm = arm_load(re)

    # ------------------ Leg Load ------------------
    avg_knee_angle = (lk + rk) / 2
    leg_factor = (180 - avg_knee_angle) / 90  # squatting = more load
    leg_load = body_weight * 0.5 * (1 + leg_factor)

    # ------------------ Hip Load ------------------
    avg_hip_angle = (lh + rh) / 2
    hip_load = body_weight * 0.3 * (1 + (180 - avg_hip_angle) / 120)

    # ------------------ Chest Load ------------------
    chest_load = body_weight * 0.2 * (1 + (180 - avg_hip_angle) / 90)

    return {
        'image': row['image'],
        'left_arm_load_kg': round(left_arm, 2),
        'right_arm_load_kg': round(right_arm, 2),
        'leg_load_kg': round(leg_load, 2),
        'hip_load_kg': round(hip_load, 2),
        'chest_load_kg': round(chest_load, 2)
    }


Apply Load Estimation to All Images

In [9]:
# -------------------------------
# ✅ Estimate and Save Loads
# -------------------------------
# Apply biomechanical estimation row by row
load_estimations = [estimate_load_distribution(row) for _, row in df_angles.iterrows()]
df_loads = pd.DataFrame(load_estimations)

print("🏋️ Biomechanical Load Estimations:")
display(df_loads.head())

# Save to /content/
df_loads.to_csv("/content/load_estimations.csv", index=False)
print("✅ Load estimations saved to /content/load_estimations.csv")

# Save to Google Drive
drive_loads_path = os.path.join(drive_folder, "load_estimations.csv")
df_loads.to_csv(drive_loads_path, index=False)
print(f"✅ Load estimations also saved to {drive_loads_path}")


🏋️ Biomechanical Load Estimations:


,image,left_arm_load_kg,right_arm_load_kg,leg_load_kg,hip_load_kg,chest_load_kg
0,jeffrey-f-lin-QXzHGRXyYHE-unsplash.jpg,6.64,6.09,30.39,18.69,12.61
1,lawrence-crayton-IsSxH3_6WlE-unsplash.jpg,8.34,12.57,NaN,41.30,32.71
2,anastase-maragos-iUzgePOoGko-unsplash.jpg,6.42,5.67,30.05,19.25,13.11
3,alonso-reyes-0HlI76m4jxU-unsplash.jpg,6.53,6.60,47.80,24.21,17.52
4,rezli-QDSAgKr1cvo-unsplash.jpg,8.52,7.67,32.52,18.82,12.73


✅ Load estimations saved to /content/load_estimations.csv
✅ Load estimations also saved to /content/drive/MyDrive/SEE_Assessment/load_estimations.csv
